In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# vod_movie.head()

##### 1. 시간에 따른 VOD 시청 분석의 경우 
###### - channel data와 vod 시청 데이터를 바탕으로 시간대별 시청 장르를 분석해서 장르를 기반으로 시간대별 컨텐츠 추천

###### - 소분류를 tmdb에서 가지고 오기

1. 시간대별 시청 장르 확인 - 영화

In [ ]:
vod_movie = pd.read_csv('./LG_data/01. 전처리/vod_movie_전처리_19.csv', encoding= 'cp949')

vod_movie.drop(columns= 'Unnamed: 0', axis= 1, inplace= True)
vod_movie.drop(vod_movie[vod_movie['genre_of_ct_cl'] == '성인'].index, axis=0, inplace=True)

In [ ]:
# 'strt_dt' 열에서 시간 정보를 추출합니다.
vod_movie['hour'] = pd.to_datetime(vod_movie['strt_dt'], format = "%Y%m%d%H%M%S").dt.hour

In [ ]:
# vod_movie['hour']

In [ ]:
def grouping_hour(hour) :
    if 0 <= hour <= 5 :
        hour = '오전 12시~오전 6시'
    elif 6 <= hour <= 8 :
        hour = '오전 6시~오전 9시'
    elif 9 <= hour <= 11 :
        hour = '오전 9시~오후 12시'
    elif 12 <= hour <= 14 :
        hour = '오후 12시~오후 3시'
    elif 15 <= hour <= 17 :
        hour = '오후 3시~오후 6시'
    elif 18 <= hour <= 20 :
        hour = '오후 6시~오후 9시'
    elif 21 <= hour <= 24 :
        hour = '오후 9시~오후 12시'
    else :
        hour = np.nan
    return hour

In [ ]:
vod_movie['hour_label'] = vod_movie['hour'].apply(grouping_hour)
vod_movie['hour_label']

In [ ]:
# 시간에 따른 데이터를 그룹화하고 각 그룹에서 'genre_of_ct_cl' 열의 값에 대한 빈도를 계산합니다.
genre_counts_by_hour = vod_movie.groupby('hour_label')['genre_of_ct_cl'].value_counts().unstack(fill_value=0)

# 각 그룹에서 장르의 비율을 계산합니다.
genre_percentage_by_hour = genre_counts_by_hour.div(genre_counts_by_hour.sum(axis=1), axis=0)

# 결과 출력
print(genre_percentage_by_hour)

In [ ]:
# genre_percentage_by_hour

In [ ]:
#  시간대별 장르가 차지하는 비율을 Pie 차트로 그리기
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

for i, (hour, data) in enumerate(genre_percentage_by_hour.iterrows()):
    ax = axes[i // 3, i % 3]
    ax.pie(data, labels=data.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'Hour {hour}')

plt.tight_layout()
plt.show()